In [1]:
import os
import torch
from torchvision.transforms import transforms as T
from seesawfacenet_pytorch.src.seesaw_models import seesaw_shareFaceNet, seesaw_shuffleFaceNet, DW_seesawFaceNetv2
from PIL import Image
import cv2
import numpy as np
import torch.nn as nn



class CustomNet(nn.Module):
    def __init__(self, model):
        super(CustomNet, self).__init__()
        self.pre_process = T.Compose([T.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
        self.backbone = model

    def forward(self,x):
        x = self.pre_process(x)
        return self.backbone(x)

def get_seesawnet_model(model, saved_dict):
    # saved_dict = torch.load("saved_models/Seesaw-shuffleFaceNet_160bs/save/model_2019-06-22-17-20_accuracy_0.9184285714285714_step_568064_final.pth", map_location="cuda")
    saved_dict = {v[7:]:k for v,k in saved_dict.items()}
    model.load_state_dict(saved_dict)
    model = model.cpu()
    model.eval()

    seesawnet_model = CustomNet(model)
    seesawnet_model.eval()

    img = Image.open("img.jpg")
    img = torch.FloatTensor(cv2.resize(np.asarray(img), (112, 112))).permute(2,0,1)
    img = img/255.
    img = img.unsqueeze(0)
    out = model(img)

    return seesawnet_model

reimplement_model_seesawnet = CustomNet(torch.jit.load("saved_models/Seesaw_shuffleFaceNet.pt", map_location="cuda")).cuda().eval()

models_configs = [("reimplementation_seesawnet",reimplement_model_seesawnet,None),
                 ("seesaw_shuffleFaceNet_160", seesaw_shuffleFaceNet.seesaw_shuffleFaceNet(512), "saved_models/Seesaw-shuffleFaceNet_160bs/save/model_2019-06-22-17-20_accuracy_0.9184285714285714_step_568064_final.pth"),
                 ("seesaw_shuffleFaceNet_192", seesaw_shuffleFaceNet.seesaw_shuffleFaceNet(512), "saved_models/Seesaw-shuffleFaceNet_192bs/save/model_2019-06-14-06-06_accuracy_0.9269999999999999_step_475328_final.pth"),
                 ("seesaw_shareFaceNet_160", seesaw_shareFaceNet.seesaw_shareFaceNet(512), "saved_models/Seesaw-shareFaceNet_160bs/save/model_2019-06-19-04-38_accuracy_0.9298571428571428_step_568064_final.pth"),
                 ("DW_seesawFaceNetv2", DW_seesawFaceNetv2.DW_seesawFaceNetv2(512), "saved_models/DW-SeesawFaceNetv2_128bs/save/model_2019-08-03-07-50_accuracy_0.9486000000000001_step_727840_final.pth")
]



In [2]:
def generate_embeds(path, model):
    img = Image.open(path)
    img = torch.FloatTensor(cv2.resize(np.asarray(img), (112, 112))).permute(2,0,1)
    img = img/255.

    return model(img.unsqueeze(0).cuda()).cpu().detach().numpy()[0]



path = "dataset/lfw_funneled"

img_names = os.listdir(path)


In [3]:
all_img_path = []

for i in os.listdir(path):
    if i[-3:] == "txt":
        continue
    for j in os.listdir(path+'/'+i):
        if "ipynb_checkpoints" in j:
            continue
        all_img_path.append([i,j])


In [4]:
from torch.utils.data import DataLoader, Dataset

class ImgGenDataset(Dataset):
  def __init__(self, x):
      self.x = x

  def __len__(self):
      return len(self.x)

  def __getitem__(self, idx):
      img = Image.open(path+'/'+'/'.join(self.x[idx]))
      img = torch.FloatTensor(cv2.resize(np.asarray(img), (112, 112))).permute(2,0,1)
      img = img/255.

      return img, self.x[idx]

imgdataset = ImgGenDataset(all_img_path)
ImgGenerator = DataLoader(imgdataset, 64)


In [5]:
next(iter(ImgGenerator))[1][1]

('Aaron_Eckhart_0001.jpg',
 'Aaron_Guiel_0001.jpg',
 'Aaron_Patterson_0001.jpg',
 'Aaron_Peirsol_0001.jpg',
 'Aaron_Peirsol_0002.jpg',
 'Aaron_Peirsol_0003.jpg',
 'Aaron_Peirsol_0004.jpg',
 'Aaron_Pena_0001.jpg',
 'Aaron_Sorkin_0001.jpg',
 'Aaron_Sorkin_0002.jpg',
 'Aaron_Tippin_0001.jpg',
 'Abbas_Kiarostami_0001.jpg',
 'Abba_Eban_0001.jpg',
 'Abdel_Aziz_Al-Hakim_0001.jpg',
 'Abdel_Madi_Shabneh_0001.jpg',
 'Abdel_Nasser_Assidi_0001.jpg',
 'Abdel_Nasser_Assidi_0002.jpg',
 'Abdoulaye_Wade_0001.jpg',
 'Abdoulaye_Wade_0002.jpg',
 'Abdoulaye_Wade_0003.jpg',
 'Abdoulaye_Wade_0004.jpg',
 'Abdulaziz_Kamilov_0001.jpg',
 'Abdullah_0001.jpg',
 'Abdullah_0002.jpg',
 'Abdullah_0003.jpg',
 'Abdullah_0004.jpg',
 'Abdullah_Ahmad_Badawi_0001.jpg',
 'Abdullah_al-Attiyah_0002.jpg',
 'Abdullah_al-Attiyah_0003.jpg',
 'Abdullah_Gul_0001.jpg',
 'Abdullah_Gul_0002.jpg',
 'Abdullah_Gul_0003.jpg',
 'Abdullah_Gul_0004.jpg',
 'Abdullah_Gul_0005.jpg',
 'Abdullah_Gul_0006.jpg',
 'Abdullah_Gul_0007.jpg',
 'Abdullah_

In [6]:
from tqdm.auto import tqdm
import gc

gc.enable()


for name,model,saved_dict_path in models_configs:
    print(name)
    if saved_dict_path == None:
        model = model

    else:
        model = get_seesawnet_model(model, torch.load(saved_dict_path))

    model = model.cuda()
    model.eval()

    if f"embeddings_{name}" not in os.listdir():
        os.mkdir(f"embeddings_{name}")

    with torch.no_grad():
        for img, path_info in tqdm(ImgGenerator):
            img = img.cuda()
            out = model(img)

            for i,j,k in zip(out.detach().cpu().numpy(), path_info[0], path_info[1]):
                if j not in os.listdir(f"embeddings_{name}"):
                    os.mkdir(f"embeddings_{name}/{j}")

                np.save(f"embeddings_{name}/{j}/{k}.npy", i)
                    
        

reimplementation_seesawnet


100%|██████████| 206/206 [02:53<00:00,  1.19it/s]


seesaw_shuffleFaceNet_160


100%|██████████| 206/206 [01:48<00:00,  1.90it/s]


seesaw_shuffleFaceNet_192


100%|██████████| 206/206 [01:45<00:00,  1.95it/s]


seesaw_shareFaceNet_160


100%|██████████| 206/206 [01:52<00:00,  1.83it/s]


DW_seesawFaceNetv2


100%|██████████| 206/206 [01:55<00:00,  1.79it/s]


In [7]:
def euclidean_dist(x1, x2):
    return np.linalg.norm(x1 - x2)


In [8]:
euclidean_dist(np.load("embeddings_reimplementation_seesawnet\Aaron_Peirsol\Aaron_Peirsol_0001.jpg.npy"), np.load("embeddings_reimplementation_seesawnet\Aaron_Peirsol\Aaron_Peirsol_0003.jpg.npy"))

54.06436

In [9]:
euclidean_dist(np.load("embeddings_reimplementation_seesawnet\Aaron_Peirsol\Aaron_Peirsol_0001.jpg.npy"), np.load("embeddings_reimplementation_seesawnet\Aaron_Patterson\Aaron_Patterson_0001.jpg.npy"))

92.5492

In [10]:
euclidean_dist(np.load("embeddings_DW_seesawFaceNetv2\Aaron_Peirsol\Aaron_Peirsol_0001.jpg.npy"), np.load("embeddings_DW_seesawFaceNetv2\Aaron_Peirsol\Aaron_Peirsol_0003.jpg.npy"))

0.8070517

In [11]:
euclidean_dist(np.load("embeddings_DW_seesawFaceNetv2\Aaron_Peirsol\Aaron_Peirsol_0001.jpg.npy"), np.load("embeddings_DW_seesawFaceNetv2\Aaron_Patterson\Aaron_Patterson_0001.jpg.npy"))

1.31352